# NoteBook To Check Point or Polygon Inside Polygon

The objective of this notebook is to search a set of points or polygons for spatial overlap with a largerset

By Cascsde Tuholske 2019-01-17

#### Notes
1. Points should be dialated to circles to prevent one falling just outside the larger polygon set (see Lusaka)

### Depedencies

In [5]:
import sys

print(sys.executable)

/usr/bin/python3


In [1]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from shapely.geometry import mapping
from shapely.geometry import Polygon
from shapely.geometry import shape
import ast
from shapely.geometry import mapping
import rasterio
import copy

### Functions

In [ ]:
def load_points (file):
    """ This function loads a csv 
    of points and turns it into shapely points"""
    df = pd.read_csv(file)

    # creating a geometry column 
    geometry = [Point(xy) for xy in zip(df['lon'], df['lat'])]

    # Coordinate reference system : WGS84
    crs = {'init': 'epsg:4326'}

    # Creating a Geographic data frame 
    point_gdf = gpd.GeoDataFrame(df, crs=crs, geometry=geometry)
    
    return point_gdf

In [ ]:
def point_buffer(gpd_df, raduis):
    "Function to make a shapely polygon buffer around a point"
   
    # AGU 2018-12-04 - radius set to ~250m at the equator 
    #radius = 250*1/(111*1000)

    # new_gpd_df = gpd.GeoDataFrame()
    gpd_df.rename(columns={'geometry':'old_geom'}, inplace=True)
    
    arr = []
    
    for point in gpd_df['old_geom']:
        buffer = point.buffer(radius)
        arr.append((buffer))
    
    gpd_df['geometry'] = arr
    
    return gpd_df

In [ ]:
def poly_point (poly, point):
    """
    This function will check if points are inside polygons if given two gpd dataframes with points and polygons
    Returns point ids, point geometry, polygon index # and polygon geometry
    """
    
    out_arr = [] #return an array <<< ---------------- ASK RYAN IF BETTER DO USE DICT 
    
    for index_point, row_point in point.iterrows():
        for index_poly, row_poly in poly.iterrows():
            if row_point['geometry'].within(row_poly['geometry']):
                point_id = row_point['id']
                point_geom = mapping(row_point['geometry']) # makes a dict w/ keys : type and cood
                poly_id = index_poly
                poly_geom = mapping(row_poly['geometry']) # makes a dict w/ keys : type and cood
                
                out_arr.append((point_id, 
                                point_geom, 
                                poly_id, 
                                poly_geom))

    return out_arr

# Note 2018-11-30 update arr to gpd_df 

In [ ]:
def poly_overlap (point_buffer, poly_raster):
    """
    This function will check if point buffers intersect with polygons 
    if given two gpd dataframes with point buffers and polygons
    Returns point ids, point geometry, polygon index #, and polygon geometry in a geopandas DF.
    It goes faster if smaller list goes first
    
    Change TOWN & CITY ARRA
    """
    
    # make arrays to fill
    country_arr = []
    city_arr = []
    osm_type_arr = []
    lat_arr = []
    lon_arr = []
    osm_id_arr = [] 
    FID_arr = [] 
    poly_geom_arr = []
    #point_geom_arr = []
    
    for index_point_buffer, row_point_buffer in point_buffer.iterrows():
        for index_poly_raster, row_poly_raster in poly_raster.iterrows():
            if row_point_buffer['geometry'].intersects(row_poly_raster['geometry']):
                country = row_point_buffer['country']
                city = row_point_buffer['town'] # Need to change town vs city
                osm_type = row_point_buffer['osm_type']
                lat = row_point_buffer['lat']
                lon = row_point_buffer['lon']
                osm_id = row_point_buffer['osm_id']
                poly_id = row_poly_raster['FID']
                poly_geom = shape(mapping(row_poly_raster['geometry'])) # make polygon
                #point_geom = row_point_buffer['old_geom']

                country_arr.append((country))
                city_arr.append((city))
                osm_type_arr.append((osm_type))
                lat_arr.append((lat))
                lon_arr.append((lon))
                osm_id_arr.append((osm_id))
                FID_arr.append((poly_id))
                poly_geom_arr.append((poly_geom))
                #point_geom_arr.append((point_geom))
    
    # put results into a geopandas df
    new_gpd_df = gpd.GeoDataFrame()
    new_gpd_df['osm_id'] = osm_id_arr
    new_gpd_df['FID'] = FID_arr
    new_gpd_df['geometry'] = poly_geom_arr
    new_gpd_df['country'] = country_arr           
    new_gpd_df['city'] = city_arr
    new_gpd_df['osm_type'] = osm_type_arr
    new_gpd_df['lat'] = lat_arr
    new_gpd_df['lon'] = lon_arr
    #new_gpd_df['point_geom'] = point_geom_arr
    
    
    return new_gpd_df

# Note 2018-11-30 update arr to gpd_df 

# OSM Merge Cities and Towns

In [ ]:
temp_data = '/Users/cascade/Github/Pop-ERL/temp_data/ERL19/'
towns = '20190430_osm_DRC_towns.csv'
cities = '20190430_osm_DRC_cities.csv'
erl_v2_data = '/Users/cascade/Github/Pop-ERL/temp_data/ERL19/ERLv2/'

towns_gdf = load_points(erl_v2_data+towns)
cities_gdf = load_points(erl_v2_data+cities)

towns_gdf['osm_type'] = 'town'
cities_gdf['osm_type'] = 'city'

print(len(towns_gdf))
print(len(cities_gdf))

osm_points = pd.concat([towns_gdf, cities_gdf], ignore_index=True, sort = False)
print(len(osm_points))

In [ ]:
osm_points.tail()

In [ ]:
# AGU 2018-12-04 - radius set to ~250m at the equator 

radius = 250*1/(111*1000)
radius

In [ ]:
osm_buffer_gdf = point_buffer(osm_points, radius)

In [ ]:
osm_buffer_gdf.head(6)

In [ ]:
osm_buffer_gdf = osm_buffer_gdf.drop(['old_geom'], axis=1)
osm_buffer_gdf.head(6)

### Analysis

In [ ]:
# Load Polygons

polygons = 'GHS_POP_GPW42000_GLOBE_R2015A_54009_1k_v1_0_Clip_1500c300'

raster_polygons = gpd.read_file(temp_data+polygons+'.shp')
print(len(raster_polygons))

In [ ]:
raster_polygons.head(6)

### Change CRS if needed

In [ ]:
raster_polygons.crs

In [ ]:
osm_buffer_gdf.crs

In [ ]:
# For GHS CRS

crs = {'ellps': 'WGS84',
 'lon_0': 0,
 'no_defs': True,
 'proj': 'moll',
 'units': 'm',
 'x_0': 0,
 'y_0': 0}

In [ ]:
raster_polygons.crs = crs

In [ ]:
raster_polygons_crs = raster_polygons.to_crs(osm_buffer_gdf.crs)

In [ ]:
# For GHS make FID and reset crs
FID = list(range(len(raster_polygons)))
raster_polygons['DN'] = FID
raster_polygons.columns.values[0] = "FID"

In [ ]:
raster_polygons.head()

### Current Start 
1. Data: 2019-01-18 @ 6:22 pm
2. Raster Poly File: GHS_POP_GPW42000_GLOBE_R2015A_54009_1k_v1_0_Clip_1500c300.shp
3. OSM Town File: 20190114_osm_africa_towns.csv
4. Clip: [25000:35000]
5. outfile - GHS_POP_GPW42000_GLOBE_R2015A_54009_1k_v1_0_Clip_1500c300_subB1.shp

In [ ]:
# run poly_overlap

import time
checkpoint = time.time()

poly_overlap_out = poly_overlap(osm_buffer_gdf, raster_polygons)

print("elapsed time is: {}s".format(time.time()-checkpoint))
print(len(poly_overlap_out))

In [ ]:
print(len(poly_overlap_out))

In [ ]:
poly_overlap_out.tail(20)

In [ ]:
poly_overlap_out.to_file(erl_v2_data+polygons+'_polyoverlap.shp', driver='ESRI Shapefile')